In [3]:
import pandas as pd
import random
import math

# read in time point 1 for training
# train on it - model 1
# test on tp 2
# read tp 2
# add it to xgb - model 2
# test on tp 3
# read tp 3
# add it to xgb - model 3
# test on tp 4
# test model 1
data = pd.read_csv('NY_trials_2002-2022_conv_agg_updated.csv')
data.head()

,State,City,Date Sown,Brand,Variety,Date of Cut,Julian Day,Plant Lodge,Plant Height,Yield % or Vernal,...,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,NY,Ithaca,5/10/02,NaN,Seedway 9558,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,Ithaca,5/10/02,NaN,HYTEST 410,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NY,Ithaca,5/10/02,NaN,Paragon BR,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NY,Ithaca,5/10/02,NaN,4A421,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NY,Ithaca,5/10/02,NaN,WL 319HQ,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def create_class(df):
    # Create a DataFrame
    print("Length of Dataset before class column: ", len(df))
    # Calculate the mean and standard deviation for the Yield column
    mean_yield = df['Yield (tons/acre)'].mean()
    std_yield = df['Yield (tons/acre)'].std()

    # Define the thresholds based on standard deviations
    lower_threshold = mean_yield - std_yield  # Yield less than -1 stdv below mean
    upper_threshold = mean_yield + std_yield  # Yield more than +1 stdv above mean

    # Classify yields based on the thresholds
    def classify_yield(yield_value):
        if yield_value < lower_threshold:
            return 0  # Low Yield (Class 1)
        elif lower_threshold <= yield_value <= upper_threshold:
            return 1  # Medium Yield (Class 2)
        else:
            return 2  # High Yield (Class 3)

    # Apply the classification function to the Yield column
    df['class'] = df['Yield (tons/acre)'].apply(classify_yield)

    # Display the result
    #print(df)
    print("Length of Dataset after: ", len(df))
    return df



In [5]:
def calculate_yield(df):
    # Load your dataset (update filename accordingly)
    print("Length of Dataset before transformation: ", len(df))
    # Ensure 'Date of Cut' is in datetime format and extract the year
    df['Date of Cut'] = pd.to_datetime(df['Date of Cut'], errors='coerce')
    df['Year'] = df['Date of Cut'].dt.year

    # Sort data by location and year (adjust columns based on your dataset structure)
    df = df.sort_values(by=['State', 'City', 'Year'])

    # Compute year-over-year differences for yield
    df['Yield_Diff'] = df.groupby(['State', 'City'])['Yield (tons/acre)'].diff()

    # Drop rows with NaN (first year will have NaN since there's no previous year)
    #df = df.dropna()

    # Save the transformed dataset
    #df.to_csv("transformed_dataset.csv", index=False)

    print("Transformation complete! Saved as 'transformed_dataset.csv'")


    # Now, create the final dataset with 'yield_diff' and other columns
    final_output = df[['State', 'City', 'Date Sown', 'Yield_Diff', 'Total Solar Radiation (W/m^2)',
                            'Total precipitation (mm)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'class', 'Date of Cut']]

    # Display the output
    #print(final_output)
    print("Length of dataset after transformation is: ", len(final_output))
    return final_output


In [6]:
def rename_columns(df):
    
    columns_to_keep=['Yield_Diff', 'Total Solar Radiation (W/m^2)', 'Total precipitation (mm)',
       'Avg Min Temp (C)', 'Avg Max Temp (C)', 'City', 'class', 'Date of Cut']

    df=df[columns_to_keep]
    #print(df.head())
    # Rename the column
    df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
    df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
    #df = df.rename(columns={'Yield_Diff':'yield'})
    #data = data.rename(columns={'Total Radiation (W/m^2)':'radiation'})
    #data = data.rename(columns={'Total Rainfall (mm)':'rain'})
    df = df.rename(columns={'Yield_Diff': 'yield'})
    df = df.rename(columns={'Avg Min Temp (C)':'avg_min_temp'})
    df = df.rename(columns={'Avg Max Temp (C)':'avg_max_temp'})
    df = df.rename(columns={'Date of Cut':'Year'})
    return df

In [7]:
def data_cleaning(df):
    columns_to_keep=['Yield (tons/acre)', 'Total Solar Radiation (W/m^2)', 'Total precipitation (mm)',
       'Avg Min Temp (C)', 'Avg Max Temp (C)', 'City', 'Date of Cut']

    df=df[columns_to_keep]
    #print(df.head())
    # Rename the column
    df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
    df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
    #df3 = df3.rename(columns={'Yield_Diff':'yield'})
    #data = data.rename(columns={'Total Radiation (W/m^2)':'radiation'})
    #data = data.rename(columns={'Total Rainfall (mm)':'rain'})
    df = df.rename(columns={'Avg Min Temp (C)':'avg_min_temp'})
    df = df.rename(columns={'Avg Max Temp (C)':'avg_max_temp'})
    df = df.rename(columns={'Date of Cut':'Year'})
    return df


In [8]:
def split_dataset_for_City(city,df):
    city_datasets = {city: city_df for city, city_df in df.groupby("City")}
    data=city_datasets.get(city)
    return data

In [9]:
# Group by 'Year' and take the mean for numeric columns
def create_consise_dataset(df):
    df_grouped = df.groupby("Year", as_index=False).mean(numeric_only=True)

    # Add back the 'City' column (if it's the same for all rows in a group)
    df_grouped["City"] = df.groupby("Year")["City"].first().values

    return df_grouped




In [10]:
def convert_date_to_year(df):
    df['Year'] = pd.to_datetime(df['Year'], errors='coerce')
    # Now extract the year
    df['Year'] = df['Year'].dt.year
    return df

In [11]:
def undersampling(df):
    min_count = max(df["Year"].value_counts().min(), 2)

    # Perform undersampling (truncate each year to match min_count)
    df_balanced = df.groupby("Year").apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)
    return df_balanced


In [12]:
import pandas as pd

def balance_data_by_year_upsample(df):
    """Ensures equal number of data points for each year by upsampling to the maximum year count."""
    
    # Check if 'Year' exists
    if "Year" not in df.columns:
        raise KeyError("The dataset does not contain a 'Year' column.")
    
    # Group by 'Year' and get the count of records per year
    year_counts = df['Year'].value_counts()
    #print(year_counts)
    # Get the maximum number of records across all years
    max_count = year_counts.max()
    #print(max_count)
    # Initialize an empty list to store balanced data
    balanced_data = []

    # Iterate over each year and sample the same number of records
    for year in year_counts.index:
        year_data = df[df['Year'] == year]
        
        # If there are fewer data points than the maximum, upsample to the max_count
        if len(year_data) < max_count:
            year_data = year_data.sample(n=max_count, replace=True, random_state=42)
            #print("Upsampling")
        # Append the sampled data to the list
        balanced_data.append(year_data)
    #print(balanced_data)
    # Concatenate all the sampled data into a single DataFrame
    balanced_df = pd.concat(balanced_data, axis=0)
    
    return balanced_df




In [13]:
print(data.head())

  State    City Date Sown  Brand       Variety Date of Cut  Julian Day  \
0    NY  Ithaca   5/10/02    NaN  Seedway 9558     6/15/05         NaN   
1    NY  Ithaca   5/10/02    NaN    HYTEST 410     6/15/05         NaN   
2    NY  Ithaca   5/10/02    NaN    Paragon BR     6/15/05         NaN   
3    NY  Ithaca   5/10/02    NaN         4A421     6/15/05         NaN   
4    NY  Ithaca   5/10/02    NaN      WL 319HQ     6/15/05         NaN   

   Plant Lodge  Plant Height  Yield % or Vernal  ...  Unnamed: 46  \
0          NaN           NaN                NaN  ...          NaN   
1          NaN           NaN                NaN  ...          NaN   
2          NaN           NaN                NaN  ...          NaN   
3          NaN           NaN                NaN  ...          NaN   
4          NaN           NaN                NaN  ...          NaN   

   Unnamed: 47  Unnamed: 48  Unnamed: 49  Unnamed: 50  Unnamed: 51  \
0          NaN          NaN          NaN          NaN          NaN   


In [14]:
data=data_cleaning(data)
#print(data.head())
data=split_dataset_for_City('Ithaca',data)
data=create_consise_dataset(data)
# Sort by 'Year' to maintain the chronological order
data=convert_date_to_year(data)
data = data.sort_values(by="Year")
#print("Before upsampling")
print("Before undersampling")
print(data.head())
#data=balance_data_by_year_upsample(data)
data=undersampling(data)
print("After Undersampling")

data=create_class(data)
data = data.sort_values(by="Year")
data.reset_index(drop=True, inplace=True)
print(len(data))
print(data.head())
#print(data.head())

Before undersampling
     Year  Yield (tons/acre)      radiation         rain  avg_min_temp  \
80   2005           1.868367  192228.728163  2104.189592      3.384498   
56   2005           2.038261  118368.720000  1340.700000      2.653400   
52   2005           1.421923  230454.950000  2535.380000      2.449000   
116  2005           1.770909  358003.820000  3721.110000      2.805700   
213  2005           0.766452   48010.950000   301.990000     12.436900   

     avg_max_temp    City  
80      14.368527  Ithaca  
56      13.607200  Ithaca  
52      13.306700  Ithaca  
116     13.793200  Ithaca  
213     25.583800  Ithaca  
After Undersampling
Length of Dataset before class column:  48
Length of Dataset after:  48
48
   Year  Yield (tons/acre)      radiation         rain  avg_min_temp  \
0  2005           0.768636  373381.000000  3837.770000      3.190400   
1  2005           2.038261  118368.720000  1340.700000      2.653400   
2  2005           1.129565  148923.230000  1512.650000 

/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1748844638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1748844638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/3914005513.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent

In [15]:
data.reset_index(drop=True, inplace=True)
print(data.head())

   Year  Yield (tons/acre)      radiation         rain  avg_min_temp  \
0  2005           0.768636  373381.000000  3837.770000      3.190400   
1  2005           2.038261  118368.720000  1340.700000      2.653400   
2  2005           1.129565  148923.230000  1512.650000      4.769600   
3  2006           0.583333  375856.040000  4160.070000      3.700100   
4  2006           1.796667  296396.879091  3098.416667      3.022491   

   avg_max_temp    City  class  
0     14.261800  Ithaca      0  
1     13.607200  Ithaca      1  
2     16.050000  Ithaca      1  
3     14.655400  Ithaca      0  
4     14.052218  Ithaca      1  


In [16]:
all_yearsDf=data

all_yearsDf=all_yearsDf.drop(columns={'Year','City'}, inplace=False)
#print(all_yearsDf.head())
all_yearsDf = all_yearsDf.rename(columns={'Yield (tons/acre)':'yield'})
print(all_yearsDf.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600     16.050000      1
3  0.583333  375856.040000  4160.070000      3.700100     14.655400      0
4  1.796667  296396.879091  3098.416667      3.022491     14.052218      1


In [17]:
# Filter out rows where 'Year' is 2022
final_yearDf = data[data['Year'] != 2022]
final_yearDf = final_yearDf.rename(columns={'Yield (tons/acre)':'yield'})
# Display the filtered DataFrame
final_yearDf=final_yearDf.drop(columns={'Year','City'}, inplace=False)
#print(final_yearDf)
print(final_yearDf.head())


      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600     16.050000      1
3  0.583333  375856.040000  4160.070000      3.700100     14.655400      0
4  1.796667  296396.879091  3098.416667      3.022491     14.052218      1


In [18]:
target_yearDf=data[data['Year'] == 2022]
target_yearDf = target_yearDf.rename(columns={'Yield (tons/acre)':'yield'})
#print(len(target_yearDf))
#print(target_yearDf.head())
# Display the filtered DataFrame
target_yearDf=target_yearDf.drop(columns={'Year','City'}, inplace=False)
#print(final_yearDf)
print(target_yearDf.head())

       yield  radiation     rain  avg_min_temp  avg_max_temp  class
45  1.466250  124268.43  1424.11        3.7791       14.9567      1
46  1.605476  333477.75  3384.09        3.2761       14.2609      1
47  1.082727  209414.53  2137.07        2.8145       13.8011      0


In [19]:
target_yearDf=target_yearDf.dropna()
all_yearsDf=all_yearsDf.dropna()
final_yearDf=final_yearDf.dropna()


In [20]:
print(target_yearDf.head())
print(all_yearsDf.head())
print(final_yearDf.head())

       yield  radiation     rain  avg_min_temp  avg_max_temp  class
45  1.466250  124268.43  1424.11        3.7791       14.9567      1
46  1.605476  333477.75  3384.09        3.2761       14.2609      1
47  1.082727  209414.53  2137.07        2.8145       13.8011      0
      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600     16.050000      1
3  0.583333  375856.040000  4160.070000      3.700100     14.655400      0
4  1.796667  296396.879091  3098.416667      3.022491     14.052218      1
      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600

In [21]:
final_yearDf.reset_index(drop=True, inplace=True)
all_yearsDf.reset_index(drop=True, inplace=True)
target_yearDf.reset_index(drop=True, inplace=True)


In [22]:
print(target_yearDf)

      yield  radiation     rain  avg_min_temp  avg_max_temp  class
0  1.466250  124268.43  1424.11        3.7791       14.9567      1
1  1.605476  333477.75  3384.09        3.2761       14.2609      1
2  1.082727  209414.53  2137.07        2.8145       13.8011      0


In [23]:
data = pd.read_csv('NY_trials_2002-2022_conv_agg_updated.csv')
data.head()

,State,City,Date Sown,Brand,Variety,Date of Cut,Julian Day,Plant Lodge,Plant Height,Yield % or Vernal,...,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,NY,Ithaca,5/10/02,NaN,Seedway 9558,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,Ithaca,5/10/02,NaN,HYTEST 410,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NY,Ithaca,5/10/02,NaN,Paragon BR,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NY,Ithaca,5/10/02,NaN,4A421,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NY,Ithaca,5/10/02,NaN,WL 319HQ,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def calculate_yield2(df):
    # Load your dataset (update filename accordingly)
    print("Length of Dataset before transformation: ", len(df))
    # Ensure 'Date of Cut' is in datetime format and extract the year
    #df['Year'] = pd.to_datetime(df['Year'], errors='coerce')
    #df['Year'] = df['Year'].dt.year

    # Sort data by location and year (adjust columns based on your dataset structure)
    df = df.sort_values(by=['City', 'Year'])

    # Compute year-over-year differences for yield
    df['Yield_Diff'] = df.groupby(['City'])['Yield (tons/acre)'].diff()

    # Drop rows with NaN (first year will have NaN since there's no previous year)
    #df = df.dropna()

    # Save the transformed dataset
    #df.to_csv("transformed_dataset.csv", index=False)

    print("Transformation complete! Saved as 'transformed_dataset.csv'")


    # Now, create the final dataset with 'yield_diff' and other columns
    final_output = df[['City', 'Yield_Diff', 'radiation',
                            'rain', 'avg_min_temp', 'avg_max_temp', 'class', 'Year']]

    # Display the output
    #print(final_output)
    print("Length of dataset after transformation is: ", len(final_output))
    return final_output


In [25]:
data=data_cleaning(data)
#print(data.head())
data=split_dataset_for_City('Ithaca',data)
#print(data.head())
data=create_consise_dataset(data)
#print(data.head())
data=convert_date_to_year(data)
#print(data.head())
data = data.sort_values(by="Year")
#data=balance_data_by_year_upsample(data)
#print(data.head())
#data=balance_data_by_year_upsample(data)
data=undersampling(data)
#print(data.head())
#print(len(data))
data=create_class(data)
#print(data.head())
data=calculate_yield2(data)
data = data.sort_values(by="Year")
print(data.head())
data.rename(columns={'Yield_Diff': 'yield'}, inplace=True)
data.drop('City', axis=1, inplace=True)

print(data.head())
print(len(data))


Length of Dataset before class column:  48
Length of Dataset after:  48
Length of Dataset before transformation:  48
Transformation complete! Saved as 'transformed_dataset.csv'
Length of dataset after transformation is:  48
     City  Yield_Diff      radiation         rain  avg_min_temp  avg_max_temp  \
0  Ithaca         NaN  373381.000000  3837.770000      3.190400     14.261800   
1  Ithaca    1.269625  118368.720000  1340.700000      2.653400     13.607200   
2  Ithaca   -0.908696  148923.230000  1512.650000      4.769600     16.050000   
3  Ithaca   -0.546232  375856.040000  4160.070000      3.700100     14.655400   
4  Ithaca    1.213333  296396.879091  3098.416667      3.022491     14.052218   

   class  Year  
0      0  2005  
1      1  2005  
2      1  2005  
3      0  2006  
4      1  2006  
      yield      radiation         rain  avg_min_temp  avg_max_temp  class  \
0       NaN  373381.000000  3837.770000      3.190400     14.261800      0   
1  1.269625  118368.720000  134

/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1748844638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1748844638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/3914005513.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent

In [26]:
print(data.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class  \
0       NaN  373381.000000  3837.770000      3.190400     14.261800      0   
1  1.269625  118368.720000  1340.700000      2.653400     13.607200      1   
2 -0.908696  148923.230000  1512.650000      4.769600     16.050000      1   
3 -0.546232  375856.040000  4160.070000      3.700100     14.655400      0   
4  1.213333  296396.879091  3098.416667      3.022491     14.052218      1   

   Year  
0  2005  
1  2005  
2  2005  
3  2006  
4  2006  


In [27]:
# List of years you want to filter
years_to_filter = [2022]

# Filter records where the 'Year' matches any of the values in the list
data_2022 = data[data['Year'].isin(years_to_filter)]

# Display the filtered records
print(len(data_2022))
print(data_2022.head())

3
       yield  radiation     rain  avg_min_temp  avg_max_temp  class  Year
46 -0.139226  124268.43  1424.11        3.7791       14.9567      1  2022
45 -0.336797  333477.75  3384.09        3.2761       14.2609      1  2022
47 -0.383523  209414.53  2137.07        2.8145       13.8011      0  2022


In [28]:
# List of years you want to filter
years_to_filter = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

# Filter records where the 'Year' matches any of the values in the list
data_no_2022 = data[data['Year'].isin(years_to_filter)]

# Display the filtered records
print(len(data_no_2022))
print(data_no_2022.head())

45
      yield      radiation         rain  avg_min_temp  avg_max_temp  class  \
0       NaN  373381.000000  3837.770000      3.190400     14.261800      0   
1  1.269625  118368.720000  1340.700000      2.653400     13.607200      1   
2 -0.908696  148923.230000  1512.650000      4.769600     16.050000      1   
3 -0.546232  375856.040000  4160.070000      3.700100     14.655400      0   
4  1.213333  296396.879091  3098.416667      3.022491     14.052218      1   

   Year  
0  2005  
1  2005  
2  2005  
3  2006  
4  2006  


In [29]:
# List of years you want to filter
years_to_filter = [2005, 2007, 2009, 2011, 2015, 2017, 2019, 2020, 2021]

# Filter records where the 'Year' matches any of the values in the list
boost_Df = data[data['Year'].isin(years_to_filter)]

# Display the filtered records
print(len(boost_Df))
print(boost_Df.head())

27
      yield  radiation     rain  avg_min_temp  avg_max_temp  class  Year
0       NaN  373381.00  3837.77        3.1904       14.2618      0  2005
1  1.269625  118368.72  1340.70        2.6534       13.6072      1  2005
2 -0.908696  148923.23  1512.65        4.7696       16.0500      1  2005
6 -0.945207  159773.14  1638.65        5.0332       16.2771      0  2007
7  1.054885  245786.57  2382.68        3.6145       14.7802      1  2007


In [30]:
data_no_2022.drop(columns=['Year'], inplace=True)
data_2022.drop(columns=['Year'], inplace=True)
boost_Df.drop(columns=['Year'], inplace=True)


/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1064651002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_2022.drop(columns=['Year'], inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1064651002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2022.drop(columns=['Year'], inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_7911/1064651002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [31]:
data_2022=data_2022.dropna()
data_no_2022=data_no_2022.dropna()
boost_Df=boost_Df.dropna()
targetDf=data_2022
data=data_no_2022
data=data.dropna()

In [32]:
# Print rows with NaN values
nan_rows = data[data.isna().any(axis=1)]

print("Rows with NaN values:")
print(nan_rows)

Rows with NaN values:
Empty DataFrame
Columns: [yield, radiation, rain, avg_min_temp, avg_max_temp, class]
Index: []


In [33]:
print(data.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
1  1.269625  118368.720000  1340.700000      2.653400     13.607200      1
2 -0.908696  148923.230000  1512.650000      4.769600     16.050000      1
3 -0.546232  375856.040000  4160.070000      3.700100     14.655400      0
4  1.213333  296396.879091  3098.416667      3.022491     14.052218      1
5 -0.318602  128832.070000  1332.090000      4.637300     15.905100      1


In [34]:
data=data.reset_index(drop=True)

In [35]:
print(data.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  1.269625  118368.720000  1340.700000      2.653400     13.607200      1
1 -0.908696  148923.230000  1512.650000      4.769600     16.050000      1
2 -0.546232  375856.040000  4160.070000      3.700100     14.655400      0
3  1.213333  296396.879091  3098.416667      3.022491     14.052218      1
4 -0.318602  128832.070000  1332.090000      4.637300     15.905100      1


In [36]:
# from sdv.tabular import TVAE

# model = TVAE()
# model.fit(data)
import numpy as np
samples_out = 20000 # total number of samples/records to generate/synthesize
no_stds = 1.5 # number of standard deviations within which synthesized values must fall
number_of_classes = (data['class'].unique()).size # number of unique classes in input data

data_len = len(data.index)
F = [] # a list of the feature vectors dataframes, one per class
for class_no in range(number_of_classes):
    df = pd.DataFrame(data[data['class'] == class_no])
    #print("In F: ")
    #print(check_nan_in_dataframe(df))
    F.append(df)


def synthesize_tabular_data(F, samples_out, no_stds, no_classes, no_records):
    new_F = []
    for index, entry in enumerate(F):
        # Clean NaN values before processing
        entry = entry.dropna(subset=['yield', 'radiation', 'rain', 'avg_max_temp', 'avg_min_temp'])

        # Check if there's data left after dropping NaN values
        if len(entry) == 0:
            print(f"Warning: Class {index} has no valid data after removing NaNs.")
            continue
        yield_ = entry['yield']
        mean_yield = yield_.mean()
        std_yield = yield_.std()
        total_rad = entry['radiation']
        mean_rad = total_rad.mean()
        std_rad = total_rad.std()
        total_rain = entry['rain']
        mean_rain = total_rain.mean()
        std_rain = total_rain.std()
        avg_max_temp = entry['avg_max_temp']
        mean_max_temp = avg_max_temp.mean()
        std_max_temp = avg_max_temp.std()
        avg_min_temp = entry['avg_min_temp']
        mean_min_temp = avg_min_temp.mean()
        std_min_temp = avg_min_temp.std()
        
        #print(str(mean_yield) + " " + str(std_yield) + " " + str(mean_rad) + " " + str(std_rad) + " " + str(mean_rain) + " " + str(std_rain) + " " + str(mean_max_temp) + " " + str(std_max_temp) + " " + str(mean_min_temp) + " " + str(std_min_temp))
        # Check for NaNs in calculated values
        if np.isnan(mean_yield) or np.isnan(std_yield):
            print(f"Warning: NaN found in 'yield' statistics for class {index}.")
            continue
        if np.isnan(mean_rad) or np.isnan(std_rad):
            print(f"Warning: NaN found in 'radiation' statistics for class {index}.")
            continue
        if np.isnan(mean_rain) or np.isnan(std_rain):
            print(f"Warning: NaN found in 'rain' statistics for class {index}.")
            continue
        if np.isnan(mean_max_temp) or np.isnan(std_max_temp):
            print(f"Warning: NaN found in 'avg_max_temp' statistics for class {index}.")
            continue
        if np.isnan(mean_min_temp) or np.isnan(std_min_temp):
            print(f"Warning: NaN found in 'avg_min_temp' statistics for class {index}.")
            continue
        
        new_yields = []
        new_rads = []
        new_rains = []
        new_max_temps = []
        new_min_temps = []
        
        # calculate potcii: percentage of this class in input
        potcii = (len(entry)/no_records)
        no_records_to_generate = round(potcii * samples_out)
        print("Number of records to generate: ", no_records_to_generate)
        for i in range(no_records_to_generate):
            new_yield = random.uniform(mean_yield - std_yield*no_stds, mean_yield + std_yield*no_stds)
            new_yields.append(new_yield)
            #print("New Yields:")
            #print(new_yields)
            
            new_rad = random.uniform(mean_rad - std_rad*no_stds, mean_rad + std_rad*no_stds)
            new_rads.append(new_rad)
            #print("Radiation:")
            #print(new_rads)
            
            new_rain = random.uniform(mean_rain - std_rain*no_stds, mean_rain + std_rain*no_stds)
            new_rains.append(new_rain)
            #print("Precipitation:")
            #print(new_rains)
        
            new_max_temp = random.uniform(mean_max_temp - std_max_temp*no_stds, mean_max_temp + std_max_temp*no_stds)
            new_max_temps.append(new_max_temp)
            #print("Max Temp:")
            #print(new_max_temps)
            
            new_min_temp = random.uniform(mean_min_temp - std_min_temp*no_stds, mean_min_temp + std_min_temp*no_stds)
            new_min_temps.append(new_min_temp)
            #print("Min Temperature")
            #print(new_min_temps)
            
        # Create a DataFrame for the new synthesized data
        new_data = pd.DataFrame({
            'yield': new_yields,
            'radiation': new_rads,
            'rain': new_rains,
            'avg_max_temp': new_max_temps,
            'avg_min_temp': new_min_temps,
            'class': [index] * len(new_yields)  # Adding the class label
        })
        
        # Concatenate the original data with the new synthesized data
        concat_data = pd.concat([entry, new_data], ignore_index=True)
        
        # Append the concatenated DataFrame to the list
        new_F.append(concat_data)
    
    # Finally, concatenate all class DataFrames into a single DataFrame
    final_data = pd.concat(new_F, ignore_index=True)
    
    return final_data
    #return pd.concat(new_F)

new_data = synthesize_tabular_data(F, samples_out, no_stds, number_of_classes, data_len)

Number of records to generate:  2727
Number of records to generate:  12727
Number of records to generate:  4545


In [37]:
# Print rows with NaN values
nan_rows = new_data[new_data.isna().any(axis=1)]

print("Rows with NaN values:")
print(nan_rows)
new_data=new_data.dropna()
print(len(new_data))

Rows with NaN values:
Empty DataFrame
Columns: [yield, radiation, rain, avg_min_temp, avg_max_temp, class]
Index: []
20043


In [38]:
# get aggregate data
#targetDataLoc = '~/ctgan/data/ts_Highmore_SD_11_stat.csv'

targetDf = data_2022 #pd.read_csv(targetDataLoc)
aggDf = new_data #pd.read_csv(aggDataLoc)
boostDataLoc = '~/ctgan/data/ts_Highmore_SD_99_04_07_08_10_stat.csv'
boostDf = boost_Df
targetDf.head()

,yield,radiation,rain,avg_min_temp,avg_max_temp,class
46,-0.139226,124268.43,1424.11,3.7791,14.9567,1
45,-0.336797,333477.75,3384.09,3.2761,14.2609,1
47,-0.383523,209414.53,2137.07,2.8145,13.8011,0


In [39]:
aggDf.to_csv("aggDf.csv", index=False)

In [40]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.neural_network import MLPClassifier # neural network for classification
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # adaboost for classification
import xgboost as xgb
# saving models
# from sklearn.externals import joblib
import joblib
import matplotlib.pyplot as plt



In [41]:
# filter out the features that will not be used by the machine learning models

xColumnsToKeep = ["radiation","rain", "avg_max_temp", "avg_min_temp"]

# the target to keep
yColumnsToKeep = ["yield"]

# get a dataframe containing the features and the targets
xDf = all_yearsDf[xColumnsToKeep]   #To train with non-stationary data
test_xDf = target_yearDf[xColumnsToKeep]
boost_xDf = boost_Df[xColumnsToKeep]

yDf = all_yearsDf[yColumnsToKeep]   #To train with non-stationary data
test_yDf = target_yearDf[yColumnsToKeep]
boost_yDf = boost_Df[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)
boost_xDf = boost_xDf.reset_index(drop=True)
boost_yDf = boost_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [42]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["mae"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['mae'],
    'model__max_depth': [5,10,25,50,100]
    }

#xgbParamGrid = {}

xgbParamGrid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.3],
    'model__max_depth': [3, 5, 7],
    'model__subsample': [0.8, 1.0],
    'model__colsample_bytree': [0.8, 1.0]
}

aModelList = [#(RandomForestRegressor(), rfParamGrid, "rfTup.pkl")]#,
              #(KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              #(SVC(), svrParamGrid, "svrTup.pkl")]#,
             #(MLPClassifier(), nnParamGrid, "nnTup.pkl")]#,
             #(LinearRegression(), linRegParamGrid, "linRegTup.pkl")]#,
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeRegressor(), dtParamGrid, "dtTup.pkl")]
             (xgb.XGBRegressor(), xgbParamGrid, "xgbTup.pkl")]

N = 4
workingDir = 'working_dir'
numFeatures = 4 # 11

In [43]:
# To check if any NaN exists in the entire DataFrame
if xDf.isna().any().any():
    print("There are NaN values in the dataset.")
# To check if any Infinity exists in the entire DataFrame
if np.isinf(xDf).any().any():
    print("There are Infinity values in the dataset.")

In [44]:
final_yearDf = final_yearDf.rename(columns={'Yield (tons/acre)':'yield'})
target_yearDf = target_yearDf.rename(columns={'Yield (tons/acre)':'yield'})
final_yearDf = final_yearDf.reset_index(drop=True)
final_yearDf = final_yearDf.reset_index(drop=True)
all_yearsDf = all_yearsDf.reset_index(drop=True)
target_yearDf = target_yearDf.reset_index(drop=True)

In [45]:
print(all_yearsDf.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600     16.050000      1
3  0.583333  375856.040000  4160.070000      3.700100     14.655400      0
4  1.796667  296396.879091  3098.416667      3.022491     14.052218      1


In [46]:
print(final_yearDf.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  0.768636  373381.000000  3837.770000      3.190400     14.261800      0
1  2.038261  118368.720000  1340.700000      2.653400     13.607200      1
2  1.129565  148923.230000  1512.650000      4.769600     16.050000      1
3  0.583333  375856.040000  4160.070000      3.700100     14.655400      0
4  1.796667  296396.879091  3098.416667      3.022491     14.052218      1


In [47]:
print(target_yearDf.head())

      yield  radiation     rain  avg_min_temp  avg_max_temp  class
0  1.466250  124268.43  1424.11        3.7791       14.9567      1
1  1.605476  333477.75  3384.09        3.2761       14.2609      1
2  1.082727  209414.53  2137.07        2.8145       13.8011      0


In [48]:

print(boost_Df.head())


      yield  radiation     rain  avg_min_temp  avg_max_temp  class
1  1.269625  118368.72  1340.70        2.6534       13.6072      1
2 -0.908696  148923.23  1512.65        4.7696       16.0500      1
6 -0.945207  159773.14  1638.65        5.0332       16.2771      0
7  1.054885  245786.57  2382.68        3.6145       14.7802      1
8  1.123226  233210.09  2291.76        3.2600       14.3162      2


In [49]:
saveMLResults(all_yearsDf, final_yearDf, target_yearDf, boost_xDf, boost_yDf, test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

NameError: name 'saveMLResults' is not defined

In [59]:
def getBestModel(N, xDf, yDf, emptyModel, paramGrid, features, metricToOptimize='mae'):
    """
    inputs: N - int - the number of times the model should be trained and evaluated.
            xDf - pandas dataframe - the rows represent the data points, the columns represent the features. These
                                         are the inputs into the model
            yDf - pandas dataframe - the rows represent the data points, there is only one column. This contains the
                                         the target values for the model.
            emptyModel - sklearn model - a valid sci-kit learn model with a 'fit' method.
            paramGrid - dictionary - the para_grid to be used with this model in a grid search. Note that each parameter name
                                     in the grid must start with 'model__' (two underscores).
            features - int or float - if int, then use SelectKBest where k='features'. If float, use SelectPercentile 
                                      where 'features' is the percentage
            testSize - float - the percentage of the data that should be used for the testing set (if method=='split')
            metricToOptimize - string - either 'mae' or 'r2'.
    outputs: avgMAE - the average mean absolute error of the model as it is evaluated N times.
             avgRSq - the average R^2 value of the model as it is evaluated N times.
             bestMAE - the mean absolute error of the best model out of the N iterations.
             bestRSq - the R^2 of the best model out of the N iterations.
             bestModel - the trained best model out of the N iterations.
             
    NOTE: This assumes the data in xDf has been standardized or normalized before being used in this function.
    """
    # initialize the outputs
    avgMAE = 0.0
    avgRSq = 0.0
    bestRSq = -9999999999.99
    bestMAE = 9999999999.99
    
    # get the input features in the correct format
    X = xDf.values
    # put the target values in the correct format
    columnName = yDf.columns[0]
    y = []

    for i in range(len(yDf.index)):  # loop through every row of the dataframe
        y.append(yDf.iloc[i, 0])

    # convert the list to a numpy array
    y = np.asarray(y)

    # make the cv settings
    cv = KFold(n_splits=N, random_state=42, shuffle=True)

    # for every fold
    for train_index, test_index in cv.split(X):
    #for train_index, test_index in zip(X[:224], X[224:]):
        
        # standardization
        standardScaler = preprocessing.StandardScaler()

        # feature selection
        if type(features) == int:
            featureSelection = SelectKBest(f_regression, k=features)
        elif type(features) == float:
            featuresPercentile = features/100.0
            featureSelection = SelectPercentile(f_regression, percentile=featuresPercentile)
        else:
            raise ValueError("The input 'features' is not an integer or a float. It should be.")

        # make a pipeline
        pipe = Pipeline(steps=[('standardization', standardScaler), 
                               ('feature selection', featureSelection), 
                               ('model', emptyModel)])

        # get the train and test data
        xTrain, xTest, yTrain, yTest = X[train_index], X[test_index], y[train_index], y[test_index]

        # do a grid search and K-fold cross validation
        numFolds = 5 # 5-Fold cross validation
        
       # make the model with optimized hyperparameters via a grid search with cross validation
        model = GridSearchCV(
             estimator=pipe,
             param_grid=paramGrid,
             cv=KFold(n_splits=numFolds, shuffle=True),
             scoring='neg_mean_absolute_error',
             return_train_score=False
         )
        
        #model = xgb.XGBRegressor()

        # fit model
        model.fit(xTrain, yTrain)
        #xgb_model.fit(xTrain, yTrain)

        # get predictions
        pred = model.predict(xTest)
        trainPred = model.predict(xTrain)
#         xgb_pred = xgb_model.predict(xTest)
#         xgb_trainPred = xgb_model.predict(xTrain)

        # find errors
        meanAbsoluteError = mean_absolute_error(yTest, pred)
        trainMeanAbsoluteError = mean_absolute_error(yTrain, trainPred)

        # find the R^2 values
        rSq = r2_score(yTest, pred)
        trainRSq = r2_score(yTrain, trainPred)
        
        # calculate accuracy
        #accuracy = accuracy_score(yTest, pred)
        
        # calculate f1 score
        #f1 = f1_score(yTest, pred, average='macro')
        
        #calculate matthews correlation coefficient
        #mcc = matthews_corrcoef(yTest, pred)

        # add the errors and R Squared to average values
        avgMAE += meanAbsoluteError
        avgRSq += rSq

        # check to see which metric should be optimized
        if metricToOptimize == 'r2':
            # check to see if any of these are the best values
            if (rSq > bestRSq):
                bestMAE = meanAbsoluteError
                bestModel = model
                bestRSq = rSq

        elif metricToOptimize == 'mae':
            # check to see if any of these are the best values
            if (meanAbsoluteError < bestMAE):
                bestMAE = meanAbsoluteError
                bestModel = model
                bestRSq = rSq

        else:
            raise ValueError("The input 'metricToOptimize' does not have a valid input. It must be 'r2' or 'mae'.")

    # divide the sums by N to get the averages
    avgMAE /= N
    avgRSq /= N
    
    ## get the features that were selected to train the best model
    
    # get all of the feature names and store in a numpy array
    features = np.asarray(list(xDf))
   
    # get a boolean list to say which features were kept
    #boolArray = bestModel.best_estimator_.named_steps['feature selection'].get_support()
    
    # get a list of which features were kept
    #featuresUsed = np.ndarray.tolist(features[boolArray])
    
    ## return the results
    return avgMAE, avgRSq, bestMAE, bestRSq, bestModel#, featuresUsed


In [58]:
def saveMLResults(all_yearsDf, final_yearDf, target_yearDf, boost_xDf, boost_yDf, xTest, yTest, N, xDf, yDf, modelList, workingDir, numFeatures, printResults=True):
    """
    inputs: N - int - the number of times the model should be trained and evaluated.
            xDf - pandas dataframe - the rows represent the data points, the columns represent the features. These
                                         are the inputs into the model
            yDf - pandas dataframe - the rows represent the data points, there is only one column. This contains the
                                         the target values for the model.
            modelList - list of tuples - each tuple takes the form of 
                        (empty sklearn model, parameter grid for sklearn's gridsearchcv, name of file to be saved).
                        The parameter grid should be a dictionary of possible parameter values for the empty model.
                        Look at sklearn's documentation for more information
           workingDir - string - the directory where the final results should be saved
           numFeatures - int or float - if int, then use SelectKBest where k='features'. If float, use SelectPercentile 
                                      where 'features' is the percentage
           printResults - boolean - if True, then also print the results. Otherwise, dont print the results
    outputs: nothing is returned, but the results are saved at the given location. A tuple is saved of the form
             (bestModel, bestFeatures, bestMAE, bestRSq, avgMAE, avgRSq). Each value means the following
             -bestModel - the best model found by 'getBestModel'. Note that this is the trained sklearn model itself
             -bestFeatures - the chosen features for the best model
             -bestMAE - the mean absolute error of the best model
             -bestRSq - the R squared value of the best model
             -avgMAE - the average mean absolute error of the model over the N iterations
             -avgRSq- the average R squared value of the model over the N iterations
    """
    # for every entry in the list
    for tup in modelList:
        model = tup[0]
        paramGrid = tup[1]
        filename = tup[2]

        # get results
#         avgMAE, avgRSq, bestMAE, bestRSq, bestModel, bestFeatures = getBestModel(N, xDf, yDf, model, paramGrid, 
        avgMAE, avgRSq, bestMAE, bestRSq, bestModel = getBestModel(N, xDf, yDf, model, paramGrid, 
                                                                                 features=numFeatures, metricToOptimize='r2')

        # convert tons to lbs to make results more readable
        #avgMAE = (round(avgMAE*2000, 3))
        #bestMAE = (round(bestMAE*2000, 3))

        # get the save location
        saveLoc = workingDir + "/" + filename

        # get model name
        stopIndex = filename.find(".")
        modelName = filename[:stopIndex]

        #save the new model over the old model if the new model has a better R^2 value
#         joblib.dump((bestModel, bestFeatures, bestMAE, bestRSq, avgMAE, avgRSq), saveLoc)
        joblib.dump((bestModel, bestMAE, bestRSq, avgMAE, avgRSq), saveLoc)
        

        # if 'printResults' is True, then print results
        if printResults:
            print("model: ", modelName)
            print("Avg MAE: ", avgMAE)
            print("Avg R squared: ", round(avgRSq, 3))
            print("Best MAE: ", bestMAE)
            print("Best R squared: ", round(bestRSq, 3))
            #print("Parameters of the best model: ", bestModel.best_params_)
            #print("Features selected by best model: ", bestFeatures)
            print(" ")
            
        m, r = actualTest(all_yearsDf, final_yearDf, target_yearDf, boost_xDf, boost_yDf, xTest, yTest, bestModel)
        
        print("non-local results:")
        print("MAE: ", m)
        print("R: ", r)
            
        #count += 1
        



In [57]:
def actualTest(all_yearsDf, final_yearDf, target_yearDf, boost_xDf, boost_yDf, xTest, yTest, model):
    #print("Length of all_yearsDf----This is used for plotting graph ", len(all_yearsDf))
    #print("Length of final_yearDf-----This is used for adjusting the predicted values: ", len(final_yearDf))
    #print("Length of target_yearDf-------This is used for comparing predicted values to true values-----these are true values ", len(target_yearDf))
    #print("boost_xDf: ",len(boost_xDf))
    #print("boost_yDf: ", len(boost_yDf))
    #print("Length of xTest: ", len(xTest))
    #print("Length of yTest: ", len(yTest))
    #print("test results on our test data: ")
    #print(yTest)
    
    #model = model.fit(boost_xDf, boost_yDf, xgb_model=model.get_booster())
    # get predictions
    pred = model.predict(xTest)
    #trainPred = model.predict(xTrain)
    #print("Length of predictions are: ", len(pred))

    adj_pred = []
    for i in range(len(pred)):
        adj_pred.append(final_yearDf['yield'][i] + pred[i])
    #print('adjusted predictions: ')
    #print(adj_pred)
    #print("Predictions are: ")
    #print(pred)
    #print("Length of adjusted predictions: ", len(adj_pred))
    adj_pred_Df = pd.DataFrame(adj_pred)
    #print(adj_pred_Df)
    adj_pred_Df=pd.DataFrame(pred)
    # find errors
    meanAbsoluteError = mean_absolute_error(target_yearDf['yield'], adj_pred_Df)

#     plt.scatter(yTest, pred, label='Truth vs Prediction')
#     p1 = max(max(pred), max(yTest['Yield (tons/acre)']))
#     p2 = min(min(pred), min(yTest['Yield (tons/acre)']))
#     plt.plot([p1, p2], [p1, p2], color='k', label='Perfect Predictions')
#     plt.xlabel('True Yields')
#     plt.ylabel('Predicted Yields')
#     plt.legend()
    #print("Plotting--------")
    pred_ts = []
    len_tsDf = len(all_yearsDf)
    #print("len_tsDf----Length of all_yearsDf----", len_tsDf)
    for i in range(len_tsDf - len(adj_pred), len_tsDf):
        pred_ts.append(i)
    #print("Length of pred_ts is: ", len(pred_ts))
    #print("This is the index calculated for plotting time-series graph: ", pred_ts)    
    pred_df = pd.DataFrame(adj_pred, index = pred_ts, columns = ['yield'])
    #print("Lengths of all_yearsDf and Lengths of pred_Df are: ", len(all_yearsDf['yield']), len(pred_df))
    #print("Head and shape of pred_Df")
    #print(pred_df.head())  # Check structure
    #print(pred_df.shape)   # Compare with all_yearsDf
    #print(all_yearsDf.index)
    plot_series(pd.Series(all_yearsDf['yield']), pred_df, labels=["y", "y_pred"])


    #plt.figure(figsize=(10, 6))

    # Plot actual yield using its index
    #plt.plot(all_yearsDf.index, all_yearsDf['yield'], label="Actual Yield", linestyle="-", color="blue")

    # Plot predicted yield using its index (from pred_df)
    #plt.plot(pred_df.index, pred_df['yield'], label="Predicted Yield", linestyle="--", color="red")

    #plt.xlabel("Index")  # The x-axis will now be the index of each DataFrame
    #plt.ylabel("Yield")
    #plt.legend()
    #plt.title("Actual vs. Predicted Yield")
    #plt.show()

    # find the R values
    r = np.corrcoef(target_yearDf['yield'], adj_pred)[0,1]
    
    # find the R^2 coefficient of determination
    # defining the variables
#     r2x = xTest.tolist()
#     r2y = yTest.tolist()

    # adding the constant term
    r2pred = sm.add_constant(adj_pred)

    # performing the regression
    # and fitting the model
    yTargetYield = target_yearDf['yield']
    result = sm.OLS(yTargetYield, r2pred).fit()

    # printing the summary table
    print(result.summary())
    print('skt mape: ' + str(round(skt_mape(yTargetYield, adj_pred_Df) * 100, 3)) + '%')
    print('skt smape: ' + str(round(skt_smape(yTargetYield, adj_pred_Df) * 100, 3)) + '%')
    print('skt mae: ' + str(round(skt_mae(yTargetYield, adj_pred_Df), 3)))
    print('skt mse: ' + str(round(skt_mse(yTargetYield, adj_pred_Df), 3)))
    print('skt rmse: ' + str(round(math.sqrt(skt_mse(yTargetYield, adj_pred_Df)), 3)) + '\n\n')
    
    #accuracy = accuracy_score(yTest, pred)
    
    return(meanAbsoluteError, r)
    

#     # add the errors and R Squared to average values
#     avgMAE += meanAbsoluteError
#     avgRSq += rSq

#     # check to see which metric should be optimized
#     if metricToOptimize == 'r2':
#         # check to see if any of these are the best values
#         if (rSq > bestRSq):
#             bestMAE = meanAbsoluteError
#             bestModel = model
#             bestRSq = rSq

#     elif metricToOptimize == 'mae':
#         # check to see if any of these are the best values
#         if (meanAbsoluteError < bestMAE):
#             bestMAE = meanAbsoluteError
#             bestModel = model
#             bestRSq = rSq

#     else:
#         raise ValueError("The input 'metricToOptimize' does not have a valid input. It must be 'r2' or 'mae'.")

In [103]:
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import joblib
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import math

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sktime.utils.plotting import plot_series

# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, matthews_corrcoef
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
import xgboost as xgb

skt_mape = MeanAbsolutePercentageError(symmetric=False)
skt_smape = MeanAbsolutePercentageError(symmetric=True)
skt_mae = MeanAbsoluteError()
skt_mse = MeanSquaredError()

In [2]:
###Upsampling



model:  xgbTup
Avg MAE:  0.5620175849683464
Avg R squared:  0.41
Best MAE:  0.5608421187074918
Best R squared:  0.417
 
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                    0.6626
Date:                Sun, 30 Mar 2025   Prob (F-statistic):              0.428
Time:                        13:14:06   Log-Likelihood:                 1.7264
No. Observations:                  17   AIC:                            0.5472
Df Residuals:                      15   BIC:                             2.214
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0324      0.088     11.674      0.000       0.844       1.221
x1            -0.0765      0.094     -0.814      0.428      -0.277       0.124
==============================================================================
Omnibus:                        0.199   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.905   Jarque-Bera (JB):                0.123
Skew:                           0.154   Prob(JB):                        0.940
Kurtosis:                       2.720   Cond. No.                         2.78
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
skt mape: 67.291%
skt smape: 82.925%
skt mae: 0.618
skt mse: 0.53
skt rmse: 0.728


non-local results:
MAE:  0.6178704736475723
R:  -0.20567927314573065

In [ ]:
#Undersampling

#k=10

model:  xgbTup
Avg MAE:  0.551655524049979
Avg R squared:  0.434
Best MAE:  0.5509496771298307
Best R squared:  0.439
 
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     2.361
Date:                Sun, 30 Mar 2025   Prob (F-statistic):              0.367
Time:                        17:50:36   Log-Likelihood:                 2.0898
No. Observations:                   3   AIC:                           -0.1796
Df Residuals:                       1   BIC:                            -1.982
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1358      0.202      5.623      0.112      -1.431       3.702
x1             0.2557      0.166      1.537      0.367      -1.859       2.370
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.087
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.368
Skew:                           0.417   Prob(JB):                        0.832
Kurtosis:                       1.500   Cond. No.                         3.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
skt mape: 50.543%
skt smape: 70.947%
skt mae: 0.658
skt mse: 0.474
skt rmse: 0.689


non-local results:
MAE:  0.6578999487511509
R:  0.8381427090580348

In [ ]:
#Undersampling

#k=5

model:  xgbTup
Avg MAE:  0.5515621629254313
Avg R squared:  0.434
Best MAE:  0.5488169445677961
Best R squared:  0.439
 
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     2.471
Date:                Sun, 30 Mar 2025   Prob (F-statistic):              0.361
Time:                        19:41:58   Log-Likelihood:                 2.1379
No. Observations:                   3   AIC:                           -0.2758
Df Residuals:                       1   BIC:                            -2.079
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1412      0.195      5.847      0.108      -1.339       3.621
x1             0.2541      0.162      1.572      0.361      -1.800       2.308
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.096
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.376
Skew:                           0.434   Prob(JB):                        0.829
Kurtosis:                       1.500   Cond. No.                         3.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
skt mape: 51.649%
skt smape: 73.79%
skt mae: 0.67
skt mse: 0.495
skt rmse: 0.704


non-local results:
MAE:  0.6702294998320136
R:  0.8437254871576833

In [ ]:
#Undersampling

#Added adj_pred_Df=pd.DataFrame(pred)

k=5


model:  xgbTup
Avg MAE:  0.5515621629254313
Avg R squared:  0.434
Best MAE:  0.5488169445677961
Best R squared:  0.439
 
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     2.471
Date:                Sun, 30 Mar 2025   Prob (F-statistic):              0.361
Time:                        15:35:06   Log-Likelihood:                 2.1379
No. Observations:                   3   AIC:                           -0.2758
Df Residuals:                       1   BIC:                            -2.079
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1412      0.195      5.847      0.108      -1.339       3.621
x1             0.2541      0.162      1.572      0.361      -1.800       2.308
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.096
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.376
Skew:                           0.434   Prob(JB):                        0.829
Kurtosis:                       1.500   Cond. No.                         3.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
skt mape: 131.011%
skt smape: 200.0%
skt mae: 1.738
skt mse: 3.047
skt rmse: 1.745


non-local results:
MAE:  1.7382544376276445
R:  0.8437254871576833

In [ ]:
#Undersampling-----criterion: mae


model:  xgbTup
Avg MAE:  0.5524303574221738
Avg R squared:  0.422
Best MAE:  0.5448859503870009
Best R squared:  0.434
 
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                     2.353
Date:                Mon, 31 Mar 2025   Prob (F-statistic):              0.368
Time:                        00:12:46   Log-Likelihood:                 2.0861
No. Observations:                   3   AIC:                           -0.1722
Df Residuals:                       1   BIC:                            -1.975
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1313      0.205      5.528      0.114      -1.469       3.732
x1             0.2589      0.169      1.534      0.368      -1.885       2.403
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.086
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.368
...

non-local results:
MAE:  0.6477484904678855
R:  0.8377032636488345
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "